# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
# 1. Gerekli kütüphaneleri yükle
import os
import json
import random
import string
from datetime import datetime
from dotenv import load_dotenv

# OpenAI client
from openai import OpenAI

# Anthropic client
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT

# Google Gemini client
import google.generativeai as genai

import gradio as gr


In [2]:
# 2. Ortam değişkenlerini yükle ve kullanılabilir modelleri seç (konsol sorusu kaldırıldı)
load_dotenv(override=True)

# API anahtarlarını oku
oai_key       = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
google_key    = os.getenv('GOOGLE_API_KEY')

# Hangi modeller hazır?
keys = {
    'openai':    oai_key,
    'anthropic': anthropic_key,
    'google':    google_key
}
available = [m for m,k in keys.items() if k]
if not available:
    raise RuntimeError("OpenAI, Anthropic veya Google için hiçbir API anahtarı bulunamadı.")

# Client nesnelerini önceden oluştur
clients = {}
if 'openai' in available:
    from openai import OpenAI
    clients['openai'] = OpenAI(api_key=oai_key)
if 'anthropic' in available:
    from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
    clients['anthropic'] = Anthropic(api_key=anthropic_key)
if 'google' in available:
    import google.generativeai as genai
    genai.configure(api_key=google_key)
    clients['google'] = genai

# 'available' ve 'clients' structures are now used in Gradio dropdown and chat()
# Dropdown choices: available
# Client lookup: client = clients[selected_model]
# MODEL string: use a mapping to model names when calling client methods
model_names = {
    'openai': 'gpt-4o-mini',
    'anthropic': 'claude-3-haiku',
    'google': 'gemini-2.0-flash'
}


In [3]:
# 3. Uçak bileti fiyatları sözlüğü (çift şehirli anahtarlarla)
ticket_prices = {
    ("istanbul", "berlin"): "${499}",
    ("istanbul", "tokyo"):  "$1400",
    ("ankara",   "berlin"):  "$550",
    ("izmir",    "tokyo"):  "$1350",
    ("antalya",  "paris"):  "$790",
    ("istanbul", "paris"):  "$899",
    # Ek rotalar
    ("ankara",   "tokyo"):  "$1200",
    ("izmir",    "berlin"):  "$600",
    ("antalya",  "london"): "$800",
    ("istanbul", "new york"): "$999",
}


In [4]:
# 4. Bilet fiyatı fonksiyonu (güvenli)
def get_ticket_price(departure_city, destination_city):
    """
    departure_city ve destination_city ikilisini kullanarak
    ticket_prices sözlüğünden bakar, yoksa tek şehre bakar.
    """
    dep = departure_city.lower()
    dest = destination_city.lower()
    # Önce çift anahtara bak
    price = ticket_prices.get((dep, dest))
    if price:
        return price
    # Çift anahtar yoksa, tek şehre de bakabilirsin
    return ticket_prices.get(dest, "Unknown")

In [5]:
# 5. PNR üretici & rezervasyon fonksiyonu (tarih destekli)
def generate_pnr():
    return ''.join(random.choices(string.ascii_uppercase + string.digits, k=6))

def make_reservation(departure_city, destination_city, travel_date):
    pnr = generate_pnr()
    try:
        fly_date = datetime.fromisoformat(travel_date)
        date_str = fly_date.strftime("%d %B %Y")
    except:
        date_str = travel_date
    return (
        f"Your reservation from {departure_city.title()} to {destination_city.title()} on {date_str} is confirmed.\n"
        f"Flight departs at 13:45 local time.\n"
        f"Reservation Code (PNR): {pnr}.\n"
        f"Seat selection will be available during check-in."
    )

In [6]:
# 6. Fonksiyon sarmalayıcıları
def get_ticket_price_fn(departure_city: str, destination_city: str) -> str:
    return get_ticket_price(departure_city, destination_city)

def make_reservation_fn(departure_city: str, destination_city: str, travel_date: str) -> str:
    return make_reservation(departure_city, destination_city, travel_date)


In [7]:
# 7. Fonksiyon tanımları (OpenAI function calling)
functions = [
    {
        "name": "get_ticket_price",
        "description": "Returns the ticket price between a departure and destination city.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."}
            },
            "required": ["departure_city", "destination_city"]
        }
    },
    {
        "name": "make_reservation",
        "description": "Reserves a flight from departure to destination city on a given date and returns reservation details.",
        "parameters": {
            "type": "object",
            "properties": {
                "departure_city": {"type": "string", "description": "Departure city."},
                "destination_city": {"type": "string", "description": "Destination city."},
                "travel_date": {"type": "string", "description": "Travel date in YYYY-MM-DD format."}
            },
            "required": ["departure_city", "destination_city", "travel_date"]
        }
    }
]


In [8]:
# 8. Sistem mesajı
system_message = (
    "You are a helpful assistant for an airline called FlightAI. "
    "Never call any functions unless both the departure city, destination city, and travel date are clearly provided. "
    "If the user mentions only destination or departure, ask for the missing details. "
    "Ask explicitly for travel date in YYYY-MM-DD format."
)

In [9]:
# 9. Ana sohbet fonksiyonu (çoklu model desteğiyle)
import json

def chat(history):
    # 1) Sistem + geçmiş
    msgs = [{"role": "system", "content": system_message}] + history

    if selected == "openai":
        # — OpenAI ile function calling —
        response = client.chat.completions.create(
            model=MODEL,
            messages=msgs,
            functions=functions
        )
        msg = response.choices[0].message

        if response.choices[0].finish_reason == "function_call" and msg.function_call:
            fc   = msg.function_call
            args = json.loads(fc.arguments)

            # Yerel fonksiyonu çalıştır
            if fc.name == "get_ticket_price":
                result = get_ticket_price_fn(**args)
            else:
                result = make_reservation_fn(**args)

            # Asistanın function_call bilgisini ve function yanıtını ekle
            msgs.append({"role":"assistant", "content":None,
                         "function_call":{"name":fc.name, "arguments":fc.arguments}})
            msgs.append({"role":"function", "name":fc.name, "content": json.dumps(result)})

            # Nihai yanıtı al
            followup = client.chat.completions.create(model=MODEL, messages=msgs)
            reply = followup.choices[0].message.content
        else:
            reply = msg.content

    elif selected == "anthropic":
        # — Anthropic Claude ile basit sohbet —
        # Tek mesaj prompt’u oluştur
        prompt = ""
        for m in msgs:
            if m["role"] == "user":
                prompt += HUMAN_PROMPT + m["content"]
            elif m["role"] == "assistant":
                prompt += AI_PROMPT    + m["content"]
        resp  = client.completions.create(model=MODEL, prompt=prompt, max_tokens=300)
        reply = resp.completion

    else:  # selected == "google"
        # — Google Gemini ile sohbet —
        # Sadece son kullanıcı girdisini kullanıyoruz
        last_user = history[-1]["content"]
        resp      = client.chat.create(model=MODEL, prompt=last_user)
        reply     = resp.candidates[0].message["content"]

    # 5) Geçmişi güncelle ve yanıtı döndür
    history.append({"role": "assistant", "content": reply})
    talker(reply)      # eğer sesli geri bildirim kullanıyorsan
    return history, None


In [10]:
# Cell 1: Imports and API setup with multi-model selection
from dotenv import load_dotenv
import os
import json
import uuid

# Clients for each provider
from openai import OpenAI
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
import google.generativeai as genai

# Load environment variables
load_dotenv(override=True)
openai_key    = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
google_key    = os.getenv('GOOGLE_API_KEY')

# Build list of available models
keys = {
    'openai':    openai_key,
    'anthropic': anthropic_key,
    'google':    google_key
}
available = [m for m,k in keys.items() if k]
if not available:
    raise RuntimeError("No API key found for OpenAI, Anthropic or Google.")
print("Available models:", available)

# Let user pick one
selected = input(f"Select model {available}: ").strip().lower()
if selected not in available:
    raise ValueError(f"'{selected}' is not available.")

api_key = keys[selected]

# Instantiate the appropriate client and set MODEL
if selected == 'openai':
    client = OpenAI(api_key=api_key)
    MODEL = "gpt-4o-mini"
elif selected == 'anthropic':
    client = Anthropic(api_key=api_key)
    MODEL = "claude-3-haiku"
elif selected == 'google':
    genai.configure(api_key=api_key)
    client = genai
    MODEL = "gemini-2.0-flash"


Available models: ['openai', 'anthropic', 'google']


Select model ['openai', 'anthropic', 'google']:  openai


In [11]:
# 1. Ortam değişkenlerini yükle ve kullanılabilir modelleri belirle
load_dotenv(override=True)

# API anahtarlarını oku
oai_key       = os.getenv('OPENAI_API_KEY')
anthropic_key = os.getenv('ANTHROPIC_API_KEY')
google_key    = os.getenv('GOOGLE_API_KEY')

# Hangi modellerin API anahtarları mevcut?
keys = {
    'openai':    oai_key,
    'anthropic': anthropic_key,
    'google':    google_key
}
available = [m for m, k in keys.items() if k]
if not available:
    raise RuntimeError("OpenAI, Anthropic veya Google için hiçbir API anahtarı bulunamadı.")

# Clients dictionary'i oluştur
clients = {}
if 'openai' in available:
    from openai import OpenAI
    clients['openai'] = OpenAI(api_key=oai_key)
if 'anthropic' in available:
    from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
    clients['anthropic'] = Anthropic(api_key=anthropic_key)
if 'google' in available:
    import google.generativeai as genai
    genai.configure(api_key=google_key)
    clients['google'] = genai

# Gradio dropdown için ve chat() içinde kullanılacak yapı
# available  -> dropdown seçeneklerine; clients[selected_model] -> client objesi
model_names = {
    'openai':    'gpt-4o-mini',
    'anthropic': 'claude-3-haiku',
    'google':    'gemini-2.0-flash'
}



Available models: ['openai', 'anthropic', 'google']


Select model ['openai', 'anthropic', 'google']:  google


In [12]:
# Cell 2: Demo Data
# Ticket prices for demonstration purposes (string and tuple keys)
ticket_prices = {
    # Single–city lookup
    "london":    "$799",
    "paris":     "$899",
    "tokyo":     "$1400",
    "berlin":    "$499",
    # Specific routes (tuple keys)
    ("istanbul", "paris"):   "$899",
    ("istanbul", "berlin"):  "$499",
    ("istanbul", "tokyo"):   "$1400",
    ("ankara",   "paris"):   "$550",
    ("ankara",   "berlin"):  "$550",
    ("ankara",   "tokyo"):   "$1200",
    ("izmir",    "tokyo"):   "$1350",
    ("izmir",    "berlin"):  "$600",
    ("antalya",  "paris"):   "$790",
    ("antalya",  "london"):  "$800",
    ("istanbul", "new york"): "$999",
}

# Flight schedule data without dates for simplicity
flight_schedule = [
    {"from": "istanbul", "to": "paris", "time": "08:30"},
    {"from": "istanbul", "to": "paris", "time": "15:45"},
    {"from": "istanbul", "to": "berlin", "time": "09:15"},
    {"from": "ankara",    "to": "paris",  "time": "11:00"},
]


In [13]:
# Cell 3: Tool Functions
def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    return ticket_prices.get(destination_city.lower(), "Unknown")

import random

def get_flight_time(departure_city, destination_city):
    dep = departure_city.lower()
    dest = destination_city.lower()
    # Tarifedeki saatleri bul
    times = [
        f["time"]
        for f in flight_schedule
        if f["from"] == dep and f["to"] == dest
    ]
    if times:
        return random.choice(times)

    # Fallback: rota olmasa bile sabit bir saat döndür
    return "12:00"

def book_flight(departure_city, destination_city, date, passenger_name):
    print(f"Tool book_flight called for {departure_city}->{destination_city} on {date} for {passenger_name}")
    time = get_flight_time(departure_city, destination_city)
    price = get_ticket_price(destination_city)
    if time.startswith("No"):
        return {"status": "error", "message": "No flights available"}
    pnr = str(uuid.uuid4())[:6].upper()
    return {
        "status": "ok",
        "departure_city": departure_city,
        "destination_city": destination_city,
        "date": date,
        "time": time,
        "price": price,
        "passenger_name": passenger_name,
        "pnr": pnr
    }

In [14]:
# 2) Modeli araca bildir (tools listesi)
response = client.chat.completions.create(
    model=MODEL,
    messages=msgs,
    functions=tools        # ← burayı tools ile değiştir
)
msg = response.choices[0].message

# 3) Eğer fonksiyon çağrısı geldiyse, dispatch et
if response.choices[0].finish_reason == "function_call" and msg.function_call:
    fc   = msg.function_call
    args = json.loads(fc.arguments)

    if fc.name == "get_ticket_price":
        result = get_ticket_price(**args)
    elif fc.name == "get_flight_time":
        result = get_flight_time(**args)
    elif fc.name == "book_flight":
        result = book_flight(**args)
    else:
        result = f"Unknown function: {fc.name}"

    # Ardından result’u mesaj geçmişine ekleyip follow-up çağrı yaptığın bloğu koru
    # …


AttributeError: module 'google.generativeai' has no attribute 'chat'

In [ ]:
artist_function = {
    "name": "artist",
    "description": "Generate a pop-art style vacation image for a given city.",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {"type": "string", "description": "Name of the city to illustrate."}
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

# Var olan tools listesine ekleyin
tools = [
    price_function,
    flight_time_function,
    book_flight_function,
    artist_function    # ← buraya ekledik
]


In [ ]:
# Cell 5: System Message
system_message = (
    "You are a FlightAI assistant. When the user provides departure, destination, and date, "
    "first call get_flight_time and get_ticket_price to fetch time and price, "
    "then ask for confirmation and passenger name. Only after confirmation, call book_flight."
)

In [ ]:
# Cell 6: Tool Call Handler
import json

def handle_tool_call(message):
    """
    message: response.choices[0].message
    """
    # 1) .function_call’dan adı ve argümanları al
    fc = message.function_call
    name = fc.name
    args = json.loads(fc.arguments)

    # 2) Gerçek fonksiyonu çalıştır
    if name == "get_ticket_price":
        result = get_ticket_price(**args)
    elif name == "get_flight_time":
        result = get_flight_time(**args)
    elif name == "book_flight":
        result = book_flight(**args)
    elif name == "artist":
        # artist() bir PIL.Image döndürüyor, onu base64’e çevirip
        from io import BytesIO
        import base64

        img = artist(**args)
        buf = BytesIO()
        img.save(buf, format="PNG")
        result = base64.b64encode(buf.getvalue()).decode()

    # 3) Asistan cevabı formatında döndür
    return {
        "role": "assistant",
        "content": result
    }


In [ ]:
# Cell 7: Chat Function (çoklu model desteği)
import json

def chat(message, history):
    # 1) Sistem + geçmiş + yeni kullanıcı mesajı
    if selected == "openai":
        messages = [{"role": "system", "content": system_message}] \
                 + history \
                 + [{"role": "user", "content": message}]
        # OpenAI ile tool‐calling
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            functions=tools
        )
        # Gelen tüm tool çağrılarını sırayla işle
        while response.choices[0].finish_reason == "tool_calls":
            tool_msg = response.choices[0].message
            messages.append(tool_msg)
            for tc in tool_msg.tool_calls:
                tool_response = handle_tool_call(tc)
                messages.append(tool_response)
            response = client.chat.completions.create(
                model=MODEL,
                messages=messages,
                functions=tools
            )
        reply = response.choices[0].message.content

    elif selected == "anthropic":
        # Anthropic Claude’e geçmiş + yeni mesajı HUMAN/AI prompt formatında ver
        prompt = ""
        for m in history:
            if m["role"] == "user":
                prompt += HUMAN_PROMPT + m["content"]
            elif m["role"] == "assistant":
                prompt += AI_PROMPT + m["content"]
        prompt += HUMAN_PROMPT + message
        resp = client.completions.create(
            model=MODEL,
            prompt=prompt,
            max_tokens=300
        )
        reply = resp.completion

    else:  # selected == "google"
        # Google Gemini’ye sadece son kullanıcı mesajını gönderiyoruz
        resp = client.chat.create(
            model=MODEL,
            prompt=message
        )
        reply = resp.candidates[0].message["content"]

    # 5) Geçmişi güncelle ve yanıtı dön
    history.append({"role": "user",      "content": message})
    history.append({"role": "assistant", "content": reply})
    return reply


In [ ]:

# Cell 8: Launch Gradio Interface
gr.ChatInterface(fn=chat, type="messages", title="FlightAI Demo").launch()

# Let's go multi-modal!!

We can use DALL-E-3, the image generation model behind GPT-4o, to make us some images

Let's put this in a function called artist.

### Price alert: each time I generate an image it costs about 4 cents - don't go crazy with images!

In [ ]:
# Some imports for handling images

import base64
from io import BytesIO
from PIL import Image

In [ ]:
from IPython.display import display

image = artist("İstanbul")
display(image)


In [ ]:
import base64
from io import BytesIO
from PIL import Image
from IPython.display import Audio, display

def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=message)

    audio_stream = BytesIO(response.content)
    output_filename = "output_audio.mp3"
    with open(output_filename, "wb") as f:
        f.write(audio_stream.read())

    # Play the generated audio
    display(Audio(output_filename, autoplay=True))

talker("Welcome to Paris")

In [ ]:
def chat(history):
    # 1) Sistem + geçmiş
    messages = [{"role": "system", "content": system_message}] + history

    if selected == "openai":
        # 2) İlk API çağrısı: tool-calling
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            functions=tools
        )
        image = None

        # 3) Eğer fonksiyon isteği geldiyse, dispatch et
        if response.choices[0].finish_reason == "function_call":
            msg = response.choices[0].message

            # Yerel tool’u çalıştır ve dönen role/content formatını al
            tool_resp = handle_tool_call(msg)

            # history’e assistant’ın function_call ve function yanıtını ekle
            messages.append({
                "role": "assistant",
                "content": None,
                "function_call": {
                    "name": msg.function_call.name,
                    "arguments": msg.function_call.arguments
                }
            })
            messages.append({
                "role": "function",
                "name": msg.function_call.name,
                "content": json.dumps(tool_resp["content"] if isinstance(tool_resp["content"], str) else tool_resp["content"])
            })

            # Eğer artist fonksiyonuysa, Base64 string’i Image’a dönüştür
            if msg.function_call.name == "artist":
                params = json.loads(msg.function_call.arguments)
                image = artist(**params)

            # 4) Follow-up çağrısı: nihai metni al
            followup = client.chat.completions.create(
                model=MODEL,
                messages=messages
            )
            reply = followup.choices[0].message.content
        else:
            # 5) Normal metin cevabı
            reply = response.choices[0].message.content
            image = None

    elif selected == "anthropic":
        # Anthropic için basit prompt-based sohbet
        prompt = ""
        for m in history:
            prompt += (HUMAN_PROMPT if m["role"]=="user" else AI_PROMPT) + m["content"]
        prompt += HUMAN_PROMPT + history[-1]["content"]
        resp = client.completions.create(model=MODEL, prompt=prompt, max_tokens=300)
        reply = resp.completion
        image = None

    else:  # selected == "google"
        # Gemini
        last_user = history[-1]["content"]
        resp = client.chat.create(model=MODEL, prompt=last_user)
        reply = resp.candidates[0].message["content"]
        image = None

    # 6) Geçmişi ve çıkışları hazırla
    history.append({"role": "assistant", "content": reply})
    talker(reply)
    return history, image


In [ ]:
with gr.Blocks() as ui:

    with gr.Row():
        # ➊ Model picker dropdown
        model_select = gr.Dropdown(choices=available, value=available[0], label="Model Seçin")

    with gr.Row():
        chatbot      = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)

    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:")

    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history):
        history += [{"role":"user", "content": message}]
        return "", history

    entry.submit(
        do_entry,
        inputs=[entry, chatbot],
        outputs=[entry, chatbot]
    ).then(
        # ➋ Pass both history and selected model into chat()
        chat,
        inputs=[chatbot, model_select],
        outputs=[chatbot, image_output]
    )

    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)
